In [1]:
import os 
import shutil
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
import sys
sys.path.append('..')
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from training.Helpers import metrics,make_model_cv,load_cv_data,take_params,test_cv_model
from tqdm import tqdm

In [2]:
#dataprep

# desktop = pathlib.Path('../../pics_dataset/')
# ids = [ [x,int(x.name.split('_')[1])] for x in  list(desktop.rglob("*.png"))]
# data = pd.DataFrame(ids,columns=['path','id'])
# data
# y = pd.read_pickle('../data/datasets/train/coding_pic.pkl')
# x = pd.read_pickle('../data/datasets/train/data_40k.pkl')
# x = x.loc[x.index.isin(data.id)]
# y = y.loc[y.index.isin(data.id)]
# _,_,y_train,y_test = train_test_split(x,y,test_size=0.1)
# len(y_train),len(y_test)
# for sample in [1000,2000,5000,8000,10000,15000,20000]:
#     _ = y_train.to_frame().groupby('hotel_cluster').apply(lambda x: x.sample(frac=(sample/len(y_train)))).droplevel(level=0)
#     os.mkdir(f'../../pics_sample_{sample}')
#     for i in y_train.unique():
#         os.mkdir(f'../../pics_sample_{sample}/{i}')
        
#     for i,d in data.loc[data.id.isin(_.index)].iterrows():
#         cat = _.loc[_.index == d.id].values[0][0]
#         #print(d['path'].name)
#         name = d['path'].name
#         shutil.copy(d['path'],f'../../pics_sample_{sample}/{cat}/{name}')
        
# for i in y.unique():
#     os.mkdir(f'../../pics_test/{i}')        
    
# for i,d in data.loc[data.id.isin(y_test.index)].iterrows():
#     cat = y_test.loc[y_test.index == d.id].values[0]
#     #print(d['path'].name)
#     name = d['path'].name
#     shutil.copy(d['path'],f'../../pics_test/{cat}/{name}')

In [2]:
pos =  {
    'dense1':[32,64,173,128,256],
    'dense2':[64,128,256,512],
    'drop2':[0.1,0],
    'dense3':[64,128,256,512,1024],
    'drop3':[0.1,0,0.2],
    'dense4':[64,128,256,512,1024],
    'drop4':[0.1,0,0.2],
    'dense5':[64,128,256],
    'dense6':[64,128,256],
    'drop6':[0.1,0,0.2],
    'bn':[0,1]
    
}


In [8]:
diff_params = pd.read_pickle('../results/best_10_params_nn.pkl')

In [9]:
diff_params

,dense1,dense2,drop2,dense3,drop3,dense4,drop4,dense5,dense6,drop6,bn
135,128,512,0.1,128,0.0,128,0.0,64,64,0.0,0
117,256,256,0.1,64,0.2,512,0.2,128,64,0.0,1
66,128,512,0.0,256,0.0,256,0.1,64,64,0.2,0
22,32,512,0.0,128,0.0,256,0.0,128,64,0.0,0
139,32,256,0.1,256,0.2,1024,0.0,64,128,0.1,0
5,256,128,0.1,1024,0.2,256,0.0,256,128,0.2,1
26,173,128,0.0,1024,0.2,512,0.2,128,256,0.0,0
10,256,128,0.0,128,0.0,128,0.2,64,256,0.1,0
129,128,128,0.0,256,0.0,1024,0.2,256,256,0.0,1
12,256,512,0.0,512,0.2,1024,0.1,128,64,0.0,1


In [12]:
results = []
history = []
for idx,params in enumerate(diff_params[:5]):
    print(f"itteration:{idx} \n")
    
    for sample_size in [1000,2000,5000,8000,10000,15000,20000,30000]:
        train_ds,val_ds = load_cv_data(sample_size)
        model = make_model_cv(params)
        h = model.fit(train_ds, batch_size=128, epochs=100, validation_data=val_ds, verbose = 0,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True,start_from_epoch=30)])
        
        h2  = {
            'val_loss':h.history['val_loss'],
            'val_accuracy':h.history['val_accuracy'],
            'loss':h.history['loss'],
            'accuracy':h.history['accuracy'],
            'sample_size' : sample_size,
        }
        
        history.append(h2)
        print('tutaj')
        res = test_cv_model(model)
        res['sample_size'] = sample_size
        res['params_id'] = idx
        results.append(res)
    
    pd.DataFrame(results).to_pickle(f'../results/res_cv_1.pkl')
    pd.DataFrame(history).to_pickle(f'../results/hist_cv_1.pkl')

itteration:0 

Found 1000 files belonging to 99 classes.
Using 700 files for training.
Using 300 files for validation.
tutaj
tutaj
Found 3250 files belonging to 99 classes.
Found 2001 files belonging to 99 classes.
Using 1401 files for training.
Using 600 files for validation.
